Pipeline for Extracting Text from Aventura Joven Books

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=c01bc3d711eb64fc9855829492b64a90c687d987b892e6769f57173ecc7787ee
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [83]:
files_dir = "/content/drive/MyDrive/capstone/"
filenames = ['Aventura Joven 01 - Persecucion - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 02 - Misterio en - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 03 - Perdidos en - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 04 - La chica de - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 05 - El fantasma - Elvira Sancho, Jordi Suris.pdf',
              'Aventura Joven 06 - El monstruo - Elvira Sancho, Jordi Suris.pdf']

In [84]:
filenames[0][:17]

'Aventura Joven 01'

In [86]:
# import parser object from tika
from tika import parser  
import unicodedata
import re

for filename in filenames:
  parsed_pdf = parser.from_file(files_dir+filename) 
  data = parsed_pdf['content'] 

  # Find beginning and end of text
  beg_idx = data.lower().index('capítulo')
  end_idx = data.lower().index('después de la lectura\n')
  text = data[beg_idx:end_idx]

  # write the initial text to a file (no preprocessing done here)
  with open(files_dir+filename[:17]+' Text No Preproc.txt', 'w') as f:
    f.write(text)

  # preprocess text
  # remove words with numbers in them, ex altura1 or hostal2 (footnote indicator)
  text_wo_ft_words = text
  for word in re.findall('[-a-zA-ZÀ-ÖØ-öø-ÿ]+»?\,?\.{0,3}[1234567890]{1,2}', text_wo_ft_words):
    word_idx = text_wo_ft_words.index(word)
    m = re.search('[-a-zA-ZÀ-ÖØ-öø-ÿ]+»?\,?\.{0,3}', word)
    stripped_word = m.group(0)
    text_wo_ft_words = text_wo_ft_words[:word_idx] + stripped_word + text_wo_ft_words[word_idx+len(word):]

  # remove words that contain '-\n' because they didn't fit on one line
  text_wo_broken_words = text_wo_ft_words
  for word in re.findall('[-a-zA-ZÀ-ÖØ-öø-ÿ]+\-\n{1,2}[-a-zØ-öø-ÿ]+', text_wo_broken_words):
    word_idx = text_wo_broken_words.index(word)
    hyphen_idx = word.index('-')
    if '-\n\n' in word:
      len_sep = 3
    else:
      len_sep = 2
    modified_word = word[:hyphen_idx] + word[hyphen_idx+len_sep:]
    text_wo_broken_words = text_wo_broken_words[:word_idx] + modified_word + text_wo_broken_words[word_idx+len(word):]

  with open(files_dir+ filename[:17] + ' Text No Ft Broken Words.txt', 'w') as f:
    f.write(text_wo_broken_words)
  

Attempt at Removing Footnote Definitions

In [ ]:
# def modify_words(regex_find, regex_modify, text, modify):
#   ''' Modifies all words in text that match the regex_find regex to match the regex_modify regex. '''
#   text_removed = text
#   for word in re.findall(regex_find, text_removed):
#     word_idx = text_removed.index(word)
    
#     if modify == 'regex':
#       m = re.search(regex_modify, word)
#       modified_word = m.group(0)
#     elif modify == 'remove':
#       rm_char_idx = word.index(regex_modify)
#       modified_word = word[:rm_char_idx] + word[rm_char_idx+len(regex_modify):]
    
#     text_removed = text_removed[:word_idx] + modified_word + text_removed[word_idx+len(word):]

#   return text_removed

In [ ]:
#text_wo_broken_words = modify_words('[-a-zA-ZÀ-ÖØ-öø-ÿ]+\-\n{1,2}[-a-zØ-öø-ÿ]+', '-\n', text_wo_ft_words, 'remove')
#print(text_wo_broken_words[800:2000])

In [ ]:
# remove the footnotes

In [ ]:
# Attempt at removing the footnotes
import re

# normalize the data to ignore special characters in Spanish
norm_data = unicodedata.normalize('NFD', data).encode('ascii', 'ignore').decode('utf-8')
norm_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')

m = re.search('[A-z]+[1234567890]{1,2}', norm_data[1880:])

m.group(0)
footnoted_words = [word.strip() for word in re.findall('[A-z]+[1234567890]', norm_data[1880:])]
footnote_defs = [word.strip() for word in re.findall('[0-9]{1,2}  [A-z]+[\s\w]*:[\w \n]*\.', norm_data[1880:])]

#print(footnoted_words)
#print(footnote_defs)
print(re.findall('([0-9]{1,2}  [A-z]+[\s\w]*:([\w ,\.:\-]+\n)*\n)', '''1  altura: Cusco esta a unos 3500 metros de altura, lo que en algunas personas 
provoca el llamado mal de altura o soroche. Los sintomas del mal de altura 
son: dolor de cabeza, mareos, trastornos estomacales y cansancio. Puede com-
batirse con pastillas, ejercicios de respiracion o mate.

2  hostal: alojamiento normalmente mas barato y personal que un hotel, aunque 
sin sus comodidades. La palabra se utiliza principalmente en medios rurales. 
'''))